In [1]:
#numpy loader method to load numpy arrays into the Pytorch ImageFolder dataset
import numpy as np

def my_numpy_loader(filename):
    return np.load(filename)

In [2]:
import os
os.chdir("../preprocessed_data_v2")
cwd = os.getcwd()
print(cwd)

D:\github\preprocessed_data_v2


In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

#Dicom dataset

train_data = torchvision.datasets.DatasetFolder(root='3D/train', transform=transforms.ToTensor(), target_transform=None, loader=my_numpy_loader, extensions="npy")


test_data = torchvision.datasets.DatasetFolder(root='3D/test', transform=transforms.ToTensor(), target_transform=None, loader=my_numpy_loader, extensions="npy")

print(train_data)
print(test_data)

Dataset DatasetFolder
    Number of datapoints: 491
    Root Location: 3D/train
    Transforms (if any): ToTensor()
    Target Transforms (if any): None
Dataset DatasetFolder
    Number of datapoints: 211
    Root Location: 3D/test
    Transforms (if any): ToTensor()
    Target Transforms (if any): None


In [5]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 4
num_classes = 2
batch_size = 100
num_workers = 4
learning_rate = 0.001

#Dicom dataset

train_data = torchvision.datasets.DatasetFolder(root='3D/train', transform=transforms.ToTensor(), target_transform=None, loader=my_numpy_loader, extensions="npy")


test_data = torchvision.datasets.DatasetFolder(root='3D/test', transform=transforms.ToTensor(), target_transform=None, loader=my_numpy_loader, extensions="npy")

print(train_data)
print(test_data)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                             batch_size=batch_size, shuffle=True)   #removed num_workers --> look up what num_workers does

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                             batch_size=batch_size, shuffle=False)

# Convolutional neural network (two convolutional layers)
"EDIT PARAMS FOR ALL 3D Methods, see documentation!"
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=5, stride=1, padding=2),  
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Dataset DatasetFolder
    Number of datapoints: 491
    Root Location: 3D/train
    Transforms (if any): ToTensor()
    Target Transforms (if any): None
Dataset DatasetFolder
    Number of datapoints: 211
    Root Location: 3D/test
    Transforms (if any): ToTensor()
    Target Transforms (if any): None


RuntimeError: Expected 5-dimensional input for 5-dimensional weight [16, 1, 5, 5, 5], but got 4-dimensional input of size [100, 64, 64, 64] instead